## Redundant Code

expected_text = """On the person’s demographics’ page click Involvements
Click new key agency
Using the drop down box to choose the type – i.e Health visitor, school, GP
Press on the blue people to search either person
Press all professionals
Click back or reset to clear the boxes, then search for the person using the “wild card” % sign
Click search and then click on the correct person, it will take you to this screen, press confirm
Press create.
Repeat the steps again to add another key agency."""

actual_text = """To add a key agency, you need to go to the person's demographics page and click on "Involvements".

Then, click on "New Key Agency".

You can also add key agencies from the "Involvements" section of the left-hand
menu. If the key agency is not already listed, you can add it manually. You can
also copy key agencies from other episodes."""

print("The expected result (from the pdf) is:",expected_text)  
print("The actual result (from the screenshot) is:",actual_text)  
print("The generated summary is:",summary)  
print("The full prompt that the AI is using:",prompt)  

print("The new prompt being used is:")  
print(prompt)

# Test Query Handler Agent

In [3]:
class TestQueryHandlerAgent:
    def __init__(self, document_processor, screenshot_comparer, summary_generator, model):
        self.document_processor = document_processor
        self.screenshot_comparer = screenshot_comparer
        self.summary_generator = summary_generator
        self.model = model

    def handle_query(self, pdf_path, screenshot_path, input_question, project):
        document_text = self.document_processor.extract_text_from_pdf(pdf_path)
        screenshot_text = self.screenshot_comparer.extract_text_from_screenshot(screenshot_path)
        comparison_summary = self.summary_generator.generate_summary(document_text, screenshot_text, input_question)
        evaluation_result = self.evaluate_summary(input_question, document_text, screenshot_text, comparison_summary, project)
        return evaluation_result

    def evaluate_summary(self, input_question, expected_text, actual_text, comparison_summary, project):
        prompt = (f"I am testing a piece of software that I have developed.\n"
        f"The software project is as follows: {project}\n"
        f"The actual result and expected result have been compared, the comparison is below:\n\n"
        f"{comparison_summary}\n"
        f"I want you to evaluate that summary (as a comparison between actual and expected results) and explain how accurate the software is\n"
        f"Include any other relevant details from the comparison in your evaluation\n"
        f"For extra context, this is the input question for the software testing:\n{input_question}\n"
        f"Please make sure to provide an 'overall' section as well for my testing report")

        evaluation = ""
        responses = self.model.generate_content(prompt, stream=True)
        for response in responses:
          evaluation += response.text

        return evaluation

# Document Processor Agent

In [4]:
!pip install pdfplumber
import pdfplumber

class DocumentProcessorAgent:
    def extract_text_from_pdf(self, pdf_path):
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
        return text

# Screenshot Comparer Agent

In [5]:
# Install Tesseract
!apt-get update
!apt-get install -y tesseract-ocr

# Install pytesseract
!pip install pytesseract

# Import libraries
from PIL import Image
import pytesseract

# Configure pytesseract to use the installed Tesseract executable
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

class ScreenshotComparerAgent:
    def extract_text_from_screenshot(self, image_path):
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

# Summary Generator Agent

In [2]:
class SummaryGeneratorAgent:
    def __init__(self, model):
        self.model = model

    def generate_summary(self, document_text, screenshot_text, input_question):
        prompt = f"""The screenshot png is the outcome of a test of the AI search
        and the pdf document contains the information that should be pulled to the search to generate the AI summary.\n
        The text from screenshot is as follows:\n{screenshot_text}\n
        The text from the pdf is as follows:\n{document_text}\n
        Can you provide a brief summary comparing how well the search worked?
        In the summary I want you to take close consideration of how well the screenshot text alligns to the pdf text
        and also if all or most of the information from the pdf text is conveyed in the screenshot text.\n
        Bear in mind the context of the following question for my software testing report.
        The question for the evaluation/summary is as follows:\n{input_question}"""

        summary = ""
        responses = self.model.generate_content(prompt, stream=True)
        for response in responses:
          summary += response.text

        return summary


# Copy To Clipboard

In [1]:
import IPython, time

def copy_to_clipboard(text):
    # JavaScript function to copy text to clipboard
    js_code = f"""
    var text = `{text}`;
    const tempInput = document.createElement('textarea');
    tempInput.value = text;
    document.body.appendChild(tempInput);
    tempInput.select();
    document.execCommand('copy');
    document.body.removeChild(tempInput);
    """
    display(IPython.display.Javascript(js_code))
    print("Text has been copied to clipboard.")

def copyOption(summary):
  option = input("\nDo you want to copy the AI summary to clipboard? (Y/N) ")
  if option.upper() == "Y":
    print("You have chosen to copy, it will now be copied")
    time.sleep(0.5)
    copy_to_clipboard(summary)
    time.sleep(0.5)
    print("The text that should be copied is:\n",summary)
  else:
    print("You have chosen not to copy, have a good day")


# Run this section directly

In [6]:
from google.colab import files

#!pip install --upgrade transformers
#!pip install --upgrade torch

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import vertexai
from vertexai.generative_models import GenerativeModel

# Authenticate the user
from google.colab import auth
import re
auth.authenticate_user()

vertexai.init(project="generalpurposeai", location="us-central1")

model = GenerativeModel(model_name="gemini-1.0-pro")

# Initialize the agents and the evaluation model
document_processor = DocumentProcessorAgent()
screenshot_comparer = ScreenshotComparerAgent()
summary_generator = SummaryGeneratorAgent(model)

# Create the TestQueryHandlerAgent with the evaluation model
test_query_handler = TestQueryHandlerAgent(document_processor, screenshot_comparer,  summary_generator, model)

print("Welcome to the Akhtar Hasan official software testing agent!")
project = input("To begin with, enter your project name/details: ")
input_question = input("\nNext enter the specific question you want to test: ")

while input_question != "":
  # Add file upload for PDF file in a while loop
  while True:
    print("\nNext please upload the associated PDF file for the test:")
    pdf_upload = files.upload()

    pdf_filename = next(iter(pdf_upload.keys())) if pdf_upload else None
    if pdf_filename == None:
      print("You did not upload a file for the PDF!\n")
      print("Was a file previously uploaded during this runtime session?")
      prevUpload = input("Enter 'Y' or 'y' to indicate this or alternatively 'N' or 'n' otherwise: ")
      if prevUpload.lower() == "y":
        break
      else:
        print("Error: Please upload a PDF file.")
    elif pdf_filename and pdf_filename.lower().endswith('.pdf'):
        pdf_path = pdf_filename
        print(f"PDF file uploaded successfully: {pdf_path}")
        break
    else:
        print("Error: Please upload a PDF file.")

  # Add file upload for screenshot in a while loop
  while True:
    print("\nNext please upload the associated PNG (screenshot) file for the test:")
    screenshot_upload = files.upload()

    screenshot_filename = next(iter(screenshot_upload.keys())) if screenshot_upload else None
    if screenshot_filename == None:
      print("You did not upload a file for the screenshot!\n")
      print("Was a file previously uploaded during this runtime session?")
      prevUpload = input("Enter 'Y' or 'y' to indicate this or alternatively 'N' or 'n' otherwise: ")
      if prevUpload.lower() == "y":
        break
      else:
        print("Error: Please upload a PDF file.")
    elif screenshot_filename and screenshot_filename.lower().endswith('.png'):
        screenshot_path = screenshot_filename
        print(f"Screenshot file uploaded successfully: {screenshot_path}")
        break
    else:
        print("Error: Please upload a PNG file.")

  print("\nPlease wait while the AI agent generates the evaluation of your software test...\n")

  aiEval = test_query_handler.handle_query(pdf_path, screenshot_path, input_question, project)
  # Remove * and #
  result = aiEval.replace('*', '').replace('#', '')
  print("\nThe result is as follows:")
  print(result)

  try:
      # Refined regex to capture everything after "Overall" till the end of the sentence or paragraph.
      # The pattern captures the word "Overall", and then looks for any text following it until the next paragraph or sentence-ending punctuation.
      match = re.search(r"(Overall.*?[.!?]+\s)|(Overall.*?\n)", result, re.DOTALL)

      if match:
          # Retrieve the first non-null group (if "Overall" was captured with different terminations)
          summary = match.group(1) if match.group(1) else match.group(2)
          summary = summary.strip()  # Strip extra whitespace
          cleaned_summary = re.sub(r'(Overall.*?\n)', '', summary)
          copyOption(cleaned_summary)
      else:
          print("\nSummary could not be extracted from the AI response.")
  except Exception as e:
      print(f"\nAn error occurred: {e}")

  input_question = input("\nNext enter another question you want to test (press enter to quit): ")
else:
  print("\nYou have chosen to quit!")
  print("Goodbye and enjoy the rest of your day :)")
  quit()

Welcome to the Akhtar Hasan official software testing agent!
To begin with, enter your project name/details: AI search for ContrOCC Training guides

Next enter the specific question you want to test: What do the buttons on the toolbar of ContrOCC do?

Next please upload the associated PDF file for the test:


Saving 1.2. Introduction to ContrOCC part 2.pdf to 1.2. Introduction to ContrOCC part 2.pdf
PDF file uploaded successfully: 1.2. Introduction to ContrOCC part 2.pdf

Next please upload the associated PNG (screenshot) file for the test:


Saving Screenshot 2024-08-08 141103.png to Screenshot 2024-08-08 141103.png
Screenshot file uploaded successfully: Screenshot 2024-08-08 141103.png

Please wait while the AI agent generates the evaluation of your software test...


The result is as follows:
 Evaluation of AI Search Software for ContrOCC Training Guides

Based on the provided comparison between the actual and expected results, the AI search software demonstrates a moderate level of accuracy in identifying the functions of the buttons on the ContrOCC toolbar.

Strengths:

 Alignment with PDF Text: The software accurately captures the functions of most of the buttons described in the PDF text, including the house icon, folder, contract tree, globe, back arrow, magnifying glass, notes, previous page, and next page buttons.
 Completeness of Major Functions: The analysis covers the functions of all the major buttons on the toolbar, providing users with a clear understanding of their primary functionalities.
 User-Friendly La

<IPython.core.display.Javascript object>

Text has been copied to clipboard.
The text that should be copied is:
 
The AI search software demonstrates potential for assisting users in understanding the ContrOCC toolbar functionality.

Next enter another question you want to test (press enter to quit): 

You have chosen to quit!
Goodbye and enjoy the rest of your day :)


## Next steps

- [x] While loop for file upload [11/9/24 at 17:25]
- [x] Handle not uploading a file and file previously uploaded [10/9/24 at 16:05]
- [x] Implement pyperclip functionality [10/9/24 at 14:35] (***Note:*** *Implemented before but fully working now*)
- [x] Remove redundant print statements [12/9/24 at 11:41]
- [x] Implement more contextual generation using project name as well [13/9/24 at 13:21] (Implemented) --- [13/9/24 at 13:35] (Fully Working)
- [x] Make into while loop for question generation [12/9/24 at 11:41] (Implemented) --- [12/9/24 at 12:41] (Fully Working)
- [x] Remove calls to Summary Generator agent [10/9/24 at 15:45]
- [x] Improve Summary Generator agent with Gemini [12/9/24 at 13:14] (Implemented) --- [12/9/24 at 13:38] (Working)
- [x] Add sleep statements to make user experience more seamless/smooth and less rushed/jagged [13/9/24 at 17:03]
- [x] Add New line characters in appropriate places to make text seem less bunched up in output [12/9/24 at 11:44]
- [x] Clean output and make copied text more user friendly [13/9/24 at 16:13] (Implemented)